## 基于Assistant API的旅游助手（使用qwen-plus模型）

- [阿里云百炼大模型服务平台](https://bailian.console.aliyun.com/#/home)

- [原文链接](https://help.aliyun.com/zh/model-studio/user-guide/assistant-api-based-travel-assistant)

## 1、数据准备和自定义function

In [1]:
## 模拟数据
destination="""
以下是一些推荐的旅游目的地：
1. 杭州（中国）
西湖：享有“人间天堂”的美誉，是中国最著名的自然景观之一。
灵隐寺：杭州最古老的寺庙之一，提供宁静的精神修养场所。
西溪湿地：天然的湿地生态区，可以近距离接触自然和野生生物。
2. 丽江（中国）
古城：保留了完好的历史建筑和纳西族文化，是一个不可多得的步行城市。
玉龙雪山：雄伟的山峰，提供滑雪和登山活动，景色壮观。
束河古镇：比丽江古城更为宁静的地方，适合品茶和静思。
3. 巴黎（法国）
埃菲尔铁塔：法国的象征，提供城市全景视角。
卢浮宫：世界上最大的艺术博物馆之一，收藏有《蒙娜丽莎》等珍贵艺术品。
凡尔赛宫：法国历史上的王宫，代表了欧洲花园和宫殿的顶峰。
4. 开普敦（南非）
桌山：通过缆车访问，顶部可俯瞰开普敦市和罗宾岛的壮丽景色。
好望角：著名的海角，为航海家提供的历史地标。
克尔斯滕博施植物园：世界上生物多样性最丰富的植物园之一。
"""


attraction = """
1. 丽江古城（大研古城）
特色：丽江古城是一个完美保存的古老纳西族城镇，以其石板路、流水和古桥而著名。无需购票即可进入，但维护古城的费用需购买一种维护费证书。
建议时间：建议早上开始您的游览以避开人群。
提示：晚上的古城特别迷人，有许多酒吧和咖啡馆可以享受。
2. 黑龙潭公园
特色：位于丽江古城北边，是欣赏玉龙雪山倒影的绝佳地点。
建议时间：上午或傍晚。
提示：入园免费，是拍照和散步的好地方。
3. 玉龙雪山
特色：这座壮观的雪山是丽江的标志性景点，您可以乘坐索道至冰川公园或牦牛坪。
建议时间：全天，但需留出足够的时间上山下山。
提示：高海拔可能引起高反，建议提前做好准备。
4. 白沙古镇
特色：相对于丽江古城，白沙古镇更加宁静，保留了更多传统纳西族文化。
建议时间：下午。
提示：可以尝试纳西族的传统手工艺品制作体验。
5. 束河古镇
特色：比丽江古城小而更为宁静，是了解纳西族文化的另一个好地点。
建议时间：晚上。
提示：束河古镇的夜生活相对平和，适合享受安静的晚餐。
"""


dining = """
餐饮建议
早餐：在丽江古城的小巷内尝试当地的酸奶和烧饵块。
午餐：在玉龙雪山下的餐厅尝试藏族或纳西族风味的菜肴。
晚餐：束河古镇有许多提供地道纳西菜的餐馆，推荐尝试纳西烤鱼。
"""


life_tips = """
旅行提示
衣着：丽江日夜温差大，即使在夏天也需携带厚外套。
高反：丽江海拔约2400米，部分人可能会有高原反应。建议游客到达后先适应一下高海拔环境，适当休息，多喝水，少进行剧烈运动。前往高海拔地区如玉龙雪山前，建议预防高原反应。
"""


local_customs = """
纳西族礼仪：

纳西族是丽江的主要民族，他们非常看重礼仪。与长辈交谈时，说话要礼貌且避免直视对方的眼睛，这是尊重的表现。
在参观纳西族家庭或参与当地活动时，接受主人提供的食物和饮料是礼貌的表现。
传统服饰：

当地人在特定节日或有重要活动时可能会穿着传统服装。游客可以租借纳西民族服饰拍照，但应保持尊重，不要对服饰进行任何不适当的处理。
节日和庆典：

参与当地节日如三月街（每年农历三月举行的大型集市和庆典）、庙会等，需遵守当地的规矩和传统，比如不大声喧哗、不随意涉足祭祀区域等。
"""


flight = """
航班号	机型	起飞时间	起飞机场	降落时间	降落机场	班期	餐食服务	票价
CA1469	32N	06:30	首都国际机场 T2	10:25	三义机场	每天	93%	¥1730 起
ZH1469	32N	06:30	首都国际机场 T2	10:25	三义机场	每天	93%	查看时价
JD5181	32Q	06:55	大兴国际机场	10:15	三义机场	每天	97%	¥630 起
CA1459	32N	16:20	首都国际机场 T2	20:15	三义机场	每天	97%	¥1150 起
ZH1459	32N	16:20	首都国际机场 T2	20:15	三义机场	每天	97%	查看时价
3U5234	737	16:55	大兴国际机场	20:05	三义机场	每天	-	¥2860 起
KN6139	737	16:55	大兴国际机场	20:05	三义机场	每天	-	查看时价
MU5716	737	16:55	大兴国际机场	20:05	三义机场	每天	-	¥900 起
3U5234	737	17:00	大兴国际机场	20:05	三义机场	每天	-	¥2860 起
KN6139	737	17:00	大兴国际机场	20:05	三义机场	每天	-	查看时价
MU5716	737	17:00	大兴国际机场	20:05	三义机场	每天	-	¥900 起
"""

## 自定义函数
def get_destination_recommendation(query):
    return destination
def get_attraction_recommendation(city):
    return attraction
def get_dining_recommendation(city):
    return dining
def get_life_tips(city):
    return life_tips
def get_local_customs(city):
    return local_customs
def get_current_date():
    return "今天"
def get_weather(location, date=""):
    if date == "":
        date = get_current_date()
    return date + "天气是" + "晴天"
def get_current_location():
    return "北京"
def get_path_recommendation(destination, start='', recommendation=False):
    return "建议通过飞机从" + start + "到" + destination + "\n以下是航班信息\n" + flight



# 可以被回调的函数放入此字典
function_mapper = {
    "天气查询": get_weather,
    "路径规划": get_path_recommendation,
    "获取目的地建议": get_destination_recommendation,
    "获取景点推荐": get_attraction_recommendation,
    "获取餐饮推荐": get_dining_recommendation,
    "获取旅行提示": get_life_tips,
    "获取当地风俗": get_local_customs,
}

## 2、创建assistant

In [2]:
import json
import sys
import os
from http import HTTPStatus

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import dashscope
dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")


from dashscope import Assistants, Messages, Runs, Threads



def create_assistant():
    # create assistant with information
    assistant = Assistants.create(
        model='qwen-plus',
        name='smart helper',
        description='一个旅游助手，可以通过用户诉求，调用天气查询，路径推荐，当地餐厅推荐等帮助用户。',
        instructions='你是一个旅游助手，可以通过调用插件解决问题。插件例如，天气查询，路径推荐，当地餐厅推荐等，当你无法回答问题时应当结合插件回复进行回答。请根据插件结果适当丰富回复内容。',
        tools=[{
            'type': 'function',
            'function': {
                'name': '天气查询',
                'description': '用于查询天气的插件和函数',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'location': {
                            'type': 'str',
                            'description': '待查询的地点'
                        },
                        'date': {
                            'type': 'str',
                            'description': '待查询的具体日期'
                        }
                    },
                    'required': ['location', 'date']
                }
            }
        },
            {
                'type': 'function',
                'function': {
                    'name': '路径规划',
                    'description': '用于推荐出行和旅游的路径规划，包含查询路线，规划起点到终点的路线。也用于推荐最近最热门的旅游行程。',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'start': {
                                'type': 'str',
                                'description': '待查询路径规划的出发点'
                            },
                            'destination': {
                                'type': 'str',
                                'description': '待查询路径规划的终点.'
                            },
                            'recommendation': {
                                'type': 'int',
                                'description': '用于控制是否随机推荐，用于在无法判断有路径规划终点时，设置为1，其他情况为0。此时会推荐最热门的旅游路线.'
                            }
                        },
                        'required': ['destination', 'start', 'recommendation']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': '获取目的地建议',
                    'description': '用于推荐最近热门的旅游目的地。',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'query': {
                                'type': 'str',
                                'description': '可能需要的信息'
                            },
                        },
                        'required': ['query']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': '获取景点推荐',
                    'description': '用于推荐指定城市的旅游景点。',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'city': {
                                'type': 'str',
                                'description': '城市名称'
                            },
                        },
                        'required': ['city']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': '获取餐饮推荐',
                    'description': '用于推荐指定城市的餐饮。',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'city': {
                                'type': 'str',
                                'description': '城市名称'
                            },
                        },
                        'required': ['city']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': '获取旅行提示',
                    'description': '用于获取指定城市的旅行注意事项。',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'city': {
                                'type': 'str',
                                'description': '城市名称'
                            },
                        },
                        'required': ['city']
                    }
                }
            },
            {
                'type': 'function',
                'function': {
                    'name': '获取当地风俗',
                    'description': '用于获取指定城市的当地风俗。',
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'city': {
                                'type': 'str',
                                'description': '城市名称'
                            },
                        },
                        'required': ['city']
                    }
                }
            },
        ],
    )

    return assistant

## 创建Thread、Message、Run，发送消息

In [3]:
def verify_status_code(res):
    if res.status_code != HTTPStatus.OK:
        sys.exit(res.status_code)

def send_message(assistant, message='查询杭州天气'):
    print(f"Query: {message}")

    # 创建Thread
    thread = Threads.create()
    print("=====thread======")
    print(thread)

    # 创建Message
    message = Messages.create(thread.id, content=message)
    
    # 创建Run
    run = Runs.create(thread.id, assistant_id=assistant.id)
    print("=====run======")
    print(run)

    # # get run statue
    # run_status = Runs.get(run.id, thread_id=thread.id)
    # print(run_status)

    # wait for run completed or requires_action
    run_status = Runs.wait(run.id, thread_id=thread.id)
    # print(run_status)

    # 如果需要调用function calling
    if run_status.required_action:

        f = run_status.required_action.submit_tool_outputs.tool_calls[0].function
        func_name = f['name']
        param = json.loads(f['arguments'])
        print("=====function======")
        print(f)
        if func_name in function_mapper:
            output = function_mapper[func_name](**param)
        else:
            output = ""

        tool_outputs = [{
            'output':
                output
        }]
        run = Runs.submit_tool_outputs(run.id,
                                       thread_id=thread.id,
                                       tool_outputs=tool_outputs)

        # should wait for run completed
        run_status = Runs.wait(run.id, thread_id=thread.id)
        # print(run_status)
        verify_status_code(run_status)

    run_status = Runs.get(run.id, thread_id=thread.id)
    print("=====run_status======")
    print(run_status)
    
    # get the thread messages.
    msgs = Messages.list(thread.id)
    
    print("运行结果:")
    for message in msgs['data'][::-1]:
        print("content: ", message['content'][0]['text']['value'])
    print("\n")


if __name__ == '__main__':
    assistant = create_assistant()
#     send_message(assistant=assistant, message="丽江天气怎么样？")

    # send_message(assistant=assistant,message="年假打算出去玩，有什么地点推荐吗")
    send_message(assistant=assistant,message="从北京去丽江怎么出行方便？")

    # send_message(assistant=assistant,message="丽江有什么好玩的地方？")
    # send_message(assistant=assistant,message="我喜欢吃，丽江有什么美食推荐吗")
    # send_message(assistant=assistant,message="去丽江还应该注意什么？")
    # send_message(assistant=assistant,message="能告诉我丽江当地有什么风俗吗")
    # send_message(assistant=assistant,message='从杭州到北京的出行推荐')

Query: 从北京去丽江怎么出行方便？
=====thread======
{'id': 'thread_c4af0f2f-9a82-4712-a6b9-fe1b79705727', 'object': 'thread', 'created_at': 1722324297700, 'metadata': {}, 'request_id': 'ee9a4212-4943-9c92-8a5f-e2b82c9044d0', 'status_code': 200}
=====run======
{'tools': [{'function': {'name': '天气查询', 'description': '用于查询天气的插件和函数', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'str', 'description': '待查询的地点'}, 'date': {'type': 'str', 'description': '待查询的具体日期'}}, 'required': ['location', 'date']}}, 'type': 'function'}, {'function': {'name': '路径规划', 'description': '用于推荐出行和旅游的路径规划，包含查询路线，规划起点到终点的路线。也用于推荐最近最热门的旅游行程。', 'parameters': {'type': 'object', 'properties': {'start': {'type': 'str', 'description': '待查询路径规划的出发点'}, 'destination': {'type': 'str', 'description': '待查询路径规划的终点.'}, 'recommendation': {'type': 'int', 'description': '用于控制是否随机推荐，用于在无法判断有路径规划终点时，设置为1，其他情况为0。此时会推荐最热门的旅游路线.'}}, 'required': ['destination', 'start', 'recommendation']}}, 'type': 'function'}, {'function': {'nam

{
    "assistant_id": "asst_feac773c-fb93-4da6-bf71-2d2e5bad3339",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1722324298590,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_1c45156c-401e-4f48-a314-0be412d6be0c",
    "instructions": "\u4f60\u662f\u4e00\u4e2a\u65c5\u6e38\u52a9\u624b\uff0c\u53ef\u4ee5\u901a\u8fc7\u8c03\u7528\u63d2\u4ef6\u89e3\u51b3\u95ee\u9898\u3002\u63d2\u4ef6\u4f8b\u5982\uff0c\u5929\u6c14\u67e5\u8be2\uff0c\u8def\u5f84\u63a8\u8350\uff0c\u5f53\u5730\u9910\u5385\u63a8\u8350\u7b49\uff0c\u5f53\u4f60\u65e0\u6cd5\u56de\u7b54\u95ee\u9898\u65f6\u5e94\u5f53\u7ed3\u5408\u63d2\u4ef6\u56de\u590d\u8fdb\u884c\u56de\u7b54\u3002\u8bf7\u6839\u636e\u63d2\u4ef6\u7ed3\u679c\u9002\u5f53\u4e30\u5bcc\u56de\u590d\u5185\u5bb9\u3002",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "02b5697a-f6fc-9451-a550-f5a00200f6fc",
    "required_action": nu

{
    "assistant_id": "asst_feac773c-fb93-4da6-bf71-2d2e5bad3339",
    "cancelled_at": null,
    "completed_at": 1722324300904,
    "created_at": 1722324298590,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_1c45156c-401e-4f48-a314-0be412d6be0c",
    "instructions": "\u4f60\u662f\u4e00\u4e2a\u65c5\u6e38\u52a9\u624b\uff0c\u53ef\u4ee5\u901a\u8fc7\u8c03\u7528\u63d2\u4ef6\u89e3\u51b3\u95ee\u9898\u3002\u63d2\u4ef6\u4f8b\u5982\uff0c\u5929\u6c14\u67e5\u8be2\uff0c\u8def\u5f84\u63a8\u8350\uff0c\u5f53\u5730\u9910\u5385\u63a8\u8350\u7b49\uff0c\u5f53\u4f60\u65e0\u6cd5\u56de\u7b54\u95ee\u9898\u65f6\u5e94\u5f53\u7ed3\u5408\u63d2\u4ef6\u56de\u590d\u8fdb\u884c\u56de\u7b54\u3002\u8bf7\u6839\u636e\u63d2\u4ef6\u7ed3\u679c\u9002\u5f53\u4e30\u5bcc\u56de\u590d\u5185\u5bb9\u3002",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "aab03bbc-c295-9b95-923d-74b7ecb4bedb",
    "required_ac

{
    "assistant_id": "asst_feac773c-fb93-4da6-bf71-2d2e5bad3339",
    "cancelled_at": null,
    "completed_at": 1722324300904,
    "created_at": 1722324298590,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_1c45156c-401e-4f48-a314-0be412d6be0c",
    "instructions": "\u4f60\u662f\u4e00\u4e2a\u65c5\u6e38\u52a9\u624b\uff0c\u53ef\u4ee5\u901a\u8fc7\u8c03\u7528\u63d2\u4ef6\u89e3\u51b3\u95ee\u9898\u3002\u63d2\u4ef6\u4f8b\u5982\uff0c\u5929\u6c14\u67e5\u8be2\uff0c\u8def\u5f84\u63a8\u8350\uff0c\u5f53\u5730\u9910\u5385\u63a8\u8350\u7b49\uff0c\u5f53\u4f60\u65e0\u6cd5\u56de\u7b54\u95ee\u9898\u65f6\u5e94\u5f53\u7ed3\u5408\u63d2\u4ef6\u56de\u590d\u8fdb\u884c\u56de\u7b54\u3002\u8bf7\u6839\u636e\u63d2\u4ef6\u7ed3\u679c\u9002\u5f53\u4e30\u5bcc\u56de\u590d\u5185\u5bb9\u3002",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "cbc82f5b-ae28-9eb7-a5aa-a17a9185efb1",
    "required_ac

{
    "assistant_id": "asst_feac773c-fb93-4da6-bf71-2d2e5bad3339",
    "cancelled_at": null,
    "completed_at": 1722324300904,
    "created_at": 1722324298590,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [],
    "id": "run_1c45156c-401e-4f48-a314-0be412d6be0c",
    "instructions": "\u4f60\u662f\u4e00\u4e2a\u65c5\u6e38\u52a9\u624b\uff0c\u53ef\u4ee5\u901a\u8fc7\u8c03\u7528\u63d2\u4ef6\u89e3\u51b3\u95ee\u9898\u3002\u63d2\u4ef6\u4f8b\u5982\uff0c\u5929\u6c14\u67e5\u8be2\uff0c\u8def\u5f84\u63a8\u8350\uff0c\u5f53\u5730\u9910\u5385\u63a8\u8350\u7b49\uff0c\u5f53\u4f60\u65e0\u6cd5\u56de\u7b54\u95ee\u9898\u65f6\u5e94\u5f53\u7ed3\u5408\u63d2\u4ef6\u56de\u590d\u8fdb\u884c\u56de\u7b54\u3002\u8bf7\u6839\u636e\u63d2\u4ef6\u7ed3\u679c\u9002\u5f53\u4e30\u5bcc\u56de\u590d\u5185\u5bb9\u3002",
    "last_error": null,
    "max_tokens": null,
    "metadata": {},
    "model": "qwen-plus",
    "object": "thread.run",
    "request_id": "00fe831c-3a72-97e7-a438-34495540b5ab",
    "required_ac

=====run_status======
{'tools': [{'function': {'name': '天气查询', 'description': '用于查询天气的插件和函数', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'str', 'description': '待查询的地点'}, 'date': {'type': 'str', 'description': '待查询的具体日期'}}, 'required': ['location', 'date']}}, 'type': 'function'}, {'function': {'name': '路径规划', 'description': '用于推荐出行和旅游的路径规划，包含查询路线，规划起点到终点的路线。也用于推荐最近最热门的旅游行程。', 'parameters': {'type': 'object', 'properties': {'start': {'type': 'str', 'description': '待查询路径规划的出发点'}, 'destination': {'type': 'str', 'description': '待查询路径规划的终点.'}, 'recommendation': {'type': 'int', 'description': '用于控制是否随机推荐，用于在无法判断有路径规划终点时，设置为1，其他情况为0。此时会推荐最热门的旅游路线.'}}, 'required': ['destination', 'start', 'recommendation']}}, 'type': 'function'}, {'function': {'name': '获取目的地建议', 'description': '用于推荐最近热门的旅游目的地。', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'str', 'description': '可能需要的信息'}}, 'required': ['query']}}, 'type': 'function'}, {'function': {'name': '获取景点推荐